# **Several Purposes**

In [2]:
import pandas as pd
import numpy as np

In [3]:
exercise_muscle_multiplier = pd.read_csv(r"C:\Users\marce\OneDrive\Escritorio\Marcelo Cruz\GYMIFY (no-code)\Datasets\Ejercicio_Musculo_Multiplicador_Cleaned.csv")

In [4]:
# Los que solo tienen un músculo los agrupo
muscles_by_exercise = exercise_muscle_multiplier.groupby('Ejercicio')['Músculo'].nunique().reset_index()
muscles_by_exercise = muscles_by_exercise[muscles_by_exercise.Músculo < 2].drop('Músculo', axis=1)
muscles_by_exercise = muscles_by_exercise.merge(exercise_muscle_multiplier[['Ejercicio','Músculo']].drop_duplicates(), on='Ejercicio', how='left')
muscles_by_exercise['Multiplicador de Activación'] = 1

exercise_muscle_multiplier = exercise_muscle_multiplier.loc[~exercise_muscle_multiplier.Ejercicio.isin(muscles_by_exercise.Ejercicio)]
exercise_muscle_multiplier = pd.concat([exercise_muscle_multiplier, muscles_by_exercise]).sort_values(by=['Ejercicio','Multiplicador de Activación'], ascending=[True, False])

# Los que estan repetidos pero tienen más de un músculo
exercise_muscle_multiplier_grouped = exercise_muscle_multiplier.groupby(['Ejercicio','Músculo'])['Multiplicador de Activación'].sum().reset_index()

exceed_activation = exercise_muscle_multiplier_grouped.groupby('Ejercicio')['Multiplicador de Activación'].sum().reset_index()
exceed_activation = exceed_activation[exceed_activation['Multiplicador de Activación'] > 1]
exercise_muscle_multiplier_grouped.loc[exercise_muscle_multiplier_grouped.Ejercicio.isin(exceed_activation.Ejercicio),'Multiplicador de Activación'] = exercise_muscle_multiplier_grouped['Multiplicador de Activación']/2

# Casos en particular
exercise_muscle_multiplier_grouped.loc[(exercise_muscle_multiplier_grouped.Ejercicio == 'Flexión de pierna acostado') & (exercise_muscle_multiplier_grouped.Músculo == 'Isquiotibiales'),'Multiplicador de Activación'] = 0.85
exercise_muscle_multiplier_grouped.loc[(exercise_muscle_multiplier_grouped.Ejercicio == 'Giro ruso con balón medicinal') & (exercise_muscle_multiplier_grouped.Músculo == 'Core'),'Multiplicador de Activación'] = 0.9

# Ordeno
exercise_muscle_multiplier_grouped = exercise_muscle_multiplier_grouped.sort_values(by=['Ejercicio','Multiplicador de Activación'], ascending=[True, False])

# Me cargo aquellos músculos que solo tienen un peso de 0.05 y se lo paso todo al principal de ese ejercicio
exercises_below_5pct = exercise_muscle_multiplier_grouped.loc[exercise_muscle_multiplier_grouped['Multiplicador de Activación'] == 0.05,'Ejercicio'].unique().tolist()

exercise_muscle_multiplier_grouped['principal'] = exercise_muscle_multiplier_grouped.groupby(['Ejercicio']).cumcount() + 1
exercise_muscle_multiplier_grouped.loc[(exercise_muscle_multiplier_grouped.Ejercicio.isin(exercises_below_5pct)) & (exercise_muscle_multiplier_grouped.principal == 1),'Multiplicador de Activación'] = exercise_muscle_multiplier_grouped.loc[(exercise_muscle_multiplier_grouped.Ejercicio.isin(exercises_below_5pct)) & (exercise_muscle_multiplier_grouped.principal == 1),'Multiplicador de Activación'] + 0.05
exercise_muscle_multiplier_grouped = exercise_muscle_multiplier_grouped[exercise_muscle_multiplier_grouped['Multiplicador de Activación'] != 0.05]
exercise_muscle_multiplier_grouped.drop('principal', axis=1, inplace=True)

# Caso en particular
exercises_below_10pct = exercise_muscle_multiplier_grouped.loc[exercise_muscle_multiplier_grouped['Multiplicador de Activación'] < 0.1,'Ejercicio'].unique().tolist()
exercise_muscle_multiplier_grouped.loc[(exercise_muscle_multiplier_grouped.Ejercicio == 'Dominadas') & (exercise_muscle_multiplier_grouped.Músculo == 'Dorsales'), 'Multiplicador de Activación'] = 0.6
exercise_muscle_multiplier_grouped.loc[(exercise_muscle_multiplier_grouped.Ejercicio == 'Dominadas') & (exercise_muscle_multiplier_grouped.Músculo == 'Bíceps'), 'Multiplicador de Activación'] = 0.2
exercise_muscle_multiplier_grouped.loc[(exercise_muscle_multiplier_grouped.Ejercicio == 'Dominadas') & (exercise_muscle_multiplier_grouped.Músculo == 'Deltoides posteriores'), 'Multiplicador de Activación'] = 0.1
exercise_muscle_multiplier_grouped.loc[(exercise_muscle_multiplier_grouped.Ejercicio == 'Dominadas') & (exercise_muscle_multiplier_grouped.Músculo == 'Espalda alta'), 'Multiplicador de Activación'] = 0.1
drop_index = exercise_muscle_multiplier_grouped.loc[(exercise_muscle_multiplier_grouped.Ejercicio == 'Dominadas') & (exercise_muscle_multiplier_grouped.Músculo == 'Core')].index
exercise_muscle_multiplier_grouped.drop(drop_index, inplace=True)

In [41]:
exercise_muscle_multiplier_grouped.to_csv(r"C:\Users\marce\OneDrive\Escritorio\Marcelo Cruz\GYMIFY (no-code)\Datasets\Ejercicio_Musculo_Multiplicador_Cleaned2.csv", index=False)

In [6]:
exerciseroles = pd.read_csv(r"C:\Users\marce\Downloads\Fitness App Datasets - Sheet43.csv")
patternmusclerol = pd.read_csv(r"C:\Users\marce\Downloads\Fitness App Datasets - PatternMuscleRol.csv")

In [12]:
ss = exerciseroles[['id_exercise','id_pattern','id_muscle_isolate']].copy()
ss_merged = ss.merge(patternmusclerol, how='left', on='id_pattern')
ss_merged['id_muscle'] = ss_merged[['id_muscle_isolate','id_muscle']].max(1)
ss_merged.drop('id_muscle_isolate', axis=1, inplace=True)

In [14]:
ss_merged.to_csv('ExerciseRoles.csv', index=False)